In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [13]:
from sklearn.preprocessing import MinMaxScaler

In [36]:
df = pd.read_csv("./labeled_data.csv")
df.head()

,Label,0,1,2,3,4,5,6,7,8,...,191834,191835,191836,191837,191838,191839,191840,191841,191842,191843
0,0,149.714737,149.718140,149.720032,149.715073,149.709106,149.704453,149.699890,149.699188,149.697052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,149.700302,149.691193,149.687744,149.688904,149.692169,149.694214,149.694397,149.697479,149.696991,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,149.646027,149.633041,149.626465,149.626328,149.632492,149.642410,149.658417,149.669891,149.681183,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,149.640610,149.646973,149.655563,149.663971,149.670441,149.674347,149.672531,149.667542,149.661957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,149.682312,149.681534,149.680725,149.674713,149.664841,149.656952,149.649673,149.642334,149.634903,...,149.679962,149.679382,149.677216,149.674072,149.668625,149.668945,149.67038,149.673447,149.674194,149.676224


In [38]:
df = df.fillna(0)
df.head()

,Label,0,1,2,3,4,5,6,7,8,...,191834,191835,191836,191837,191838,191839,191840,191841,191842,191843
0,0,149.714737,149.718140,149.720032,149.715073,149.709106,149.704453,149.699890,149.699188,149.697052,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1,0,149.700302,149.691193,149.687744,149.688904,149.692169,149.694214,149.694397,149.697479,149.696991,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,0,149.646027,149.633041,149.626465,149.626328,149.632492,149.642410,149.658417,149.669891,149.681183,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
3,0,149.640610,149.646973,149.655563,149.663971,149.670441,149.674347,149.672531,149.667542,149.661957,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
4,0,149.682312,149.681534,149.680725,149.674713,149.664841,149.656952,149.649673,149.642334,149.634903,...,149.679962,149.679382,149.677216,149.674072,149.668625,149.668945,149.67038,149.673447,149.674194,149.676224


In [76]:
from PIL import Image

imagedf = df.drop(columns=['Label'])
images = imagedf.values.reshape(-1, 438, 438)

scaler = MinMaxScaler()

def resize_image(image, new_size):
    normalized = scaler.fit_transform(image)
    img = Image.fromarray((normalized * 255).astype('uint8'), 'L')
    resized = img.resize(new_size, Image.LANCZOS)
    resized_array = np.array(resized).astype(float) / 255
    return scaler.inverse_transform(resized_array)

resized_images = np.array([resize_image_float(img, (40, 40)) for img in images])
resized_flat = resized_images.reshape(resized_images.shape[0], -1)
resized_df = pd.DataFrame(resized_flat)
resized_df.head()

size = 40

In [77]:
X = np.array(resized_df)
Y = np.array(df['Label'])

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, stratify=Y, random_state=42
)

In [78]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

X_train_scaled = X_train_scaled.reshape(-1, size, size, 1)
X_test_scaled = X_test_scaled.reshape(-1, size, size, 1)

In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [79]:
input_shape = (size, size, 1)

model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ])

model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [83]:
#early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(X_train_scaled, Y_train,
          batch_size=16,
          epochs=50,
          validation_split = 0.2,
          #callbacks=[early_stop],
          verbose=1)

Train on 264 samples, validate on 67 samples
Epoch 1/50
264/264 [==============================] - 1s 2ms/sample - loss: 0.3802 - acc: 0.7992 - val_loss: 0.4644 - val_acc: 0.7313
Epoch 2/50
264/264 [==============================] - 1s 2ms/sample - loss: 0.4586 - acc: 0.7955 - val_loss: 1.0585 - val_acc: 0.5970
Epoch 3/50
264/264 [==============================] - 1s 2ms/sample - loss: 0.3370 - acc: 0.8106 - val_loss: 1.3318 - val_acc: 0.5821
Epoch 4/50
264/264 [==============================] - 1s 2ms/sample - loss: 0.2935 - acc: 0.8523 - val_loss: 1.1950 - val_acc: 0.6119
Epoch 5/50
264/264 [==============================] - 1s 2ms/sample - loss: 0.3392 - acc: 0.8333 - val_loss: 0.3191 - val_acc: 0.9104
Epoch 6/50
264/264 [==============================] - 1s 2ms/sample - loss: 0.2887 - acc: 0.8712 - val_loss: 0.4560 - val_acc: 0.7463
Epoch 7/50
264/264 [==============================] - 1s 2ms/sample - loss: 0.3465 - acc: 0.8333 - val_loss: 0.2906 - val_acc: 0.9403
Epoch 8/50
264/26